#### Comic Vine API Demonstration

##### First
Some modules and loading in an API key which can be created at the [Comic Vine API page](https://comicvine.gamespot.com/api/). 

In [279]:
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.formula.api as smf
import numpy as np

In [280]:
cv_key = pd.read_csv('comicvine key.txt')
cv_key.keys()

Index(['API-key'], dtype='object')

In [281]:
key = cv_key['API-key'].iloc[0]

##### Second

I am just going to create a few variables to hold the api base URL and structure the parameter that holds your API key.

In [282]:
api_url = 'https://comicvine.gamespot.com/api/'
key_url = 'api_key='+str(key)

In [283]:
failed_request = characters_response = requests.get(api_url + 'characters/?' + key_url + '&format=json')
failed_request.status_code

403

In [284]:
with open('comicvine_out.html', 'w') as t:
    t.write(failed_request.text)

##### Explaining the solution:
From the output html file, scroll down to the bottom and you will find the following text that explains the reason for the 403 status code:

>... Or you're running a bot that does not provide a unique user agent.
> 		<p>Please provide a UNIQUE user agent that describes you. Do not use a default user agent like "PHP", "Java", "Ruby", "wget", "curl" etc.<p>
> 		You MUST provide a UNIQUE user agent. ...and for God's sake don't impersonate another bot like Google Bot that will for sure 
> 		get you permanently banned.

This just means that header information is required for the API to return data. In this case, the header does not require the API key, but instead just needs what is called a User Agent.

This User Agent can take many forms. What I include below is a default User Agent which is not entirely what this API wants, but it works well enough for this demonstration.

To find your unique user agent, open any browser and go to this [helpful header utility](https://www.whatismybrowser.com/detect/what-is-my-user-agent/).
The top box starting with something like: `Mozilla/5.0 (Windows...` is what you could use for the `User-Agent` headers parameter instead of the default used in the example below.

In [285]:
characters_response = requests.get(api_url + 'characters/?' + key_url + '&format=json', headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.0.1 Safari/605.1.15'})

In [286]:
characters_response.status_code

200

In [287]:
cha_dict = json.loads(characters_response.text)
cha_dict.keys()

dict_keys(['error', 'limit', 'offset', 'number_of_page_results', 'number_of_total_results', 'status_code', 'results', 'version'])

In [288]:
len(cha_dict['results'])

100

In [289]:
cha_dict['results'][0]

{'aliases': 'Garth Ranzz\nLightning Boy\nProty\nLive Wire\nStarfinger\nLightning Man',
 'api_detail_url': 'https://comicvine.gamespot.com/api/character/4005-1253/',
 'birth': None,
 'count_of_issue_appearances': 1055,
 'date_added': '2008-06-06 11:27:37',
 'date_last_updated': '2021-04-02 20:00:03',
 'deck': 'Garth Ranzz is one of the three original members/founders of the Legion of Super-Heroes. Empowered with the ability to control electricity, he is called Lightning Lad. Garth is the older twin brother of fellow Legionnaire Lightning Lass, and the younger brother of Lightning Lord.',
 'description': '<h2>Origin</h2><figure data-align="right" data-img-src="https://static.comicvine.com/uploads/original/0/3664/198287-189808-lightning-lad.jpg" data-ref-id="1300-198287" data-size="medium" data-ratio="1.2105263157895" data-width="228" data-embed-type="image" style="width: 228px"><a class="fluid-height" style="padding-bottom:121.1%" href="https://static.comicvine.com/uploads/original/0/366

In [290]:
cha_df = pd.DataFrame(cha_dict['results'])
cha_df.head()

,aliases,api_detail_url,birth,count_of_issue_appearances,date_added,date_last_updated,deck,description,first_appeared_in_issue,gender,id,image,name,origin,publisher,real_name,site_detail_url
0,Garth Ranzz\nLightning Boy\nProty\nLive Wire\n...,https://comicvine.gamespot.com/api/character/4...,None,1055,2008-06-06 11:27:37,2021-04-02 20:00:03,Garth Ranzz is one of the three original membe...,"<h2>Origin</h2><figure data-align=""right"" data...",{'api_detail_url': 'https://comicvine.gamespot...,1,1253,{'icon_url': 'https://comicvine.gamespot.com/a...,Lightning Lad,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Garth Ranzz,https://comicvine.gamespot.com/lightning-lad/4...
1,Nura Nal\nDreamy\nDreamer\nMiss Terious\nNura ...,https://comicvine.gamespot.com/api/character/4...,None,452,2008-06-06 11:27:38,2024-08-07 08:40:42,"Nura Nal is from the planet Naltor, where all ...","<h2>Origin</h2><figure data-align=""right"" data...",{'api_detail_url': 'https://comicvine.gamespot...,2,1254,{'icon_url': 'https://comicvine.gamespot.com/a...,Dream Girl,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Nura Nal,https://comicvine.gamespot.com/dream-girl/4005...
2,Querl Dox\nBrainiac 5.1\nBrainy\nBrainiac \nB-...,https://comicvine.gamespot.com/api/character/4...,None,1136,2008-06-06 11:27:37,2020-11-19 15:25:09,Brainiac 5 is a long-standing member of the Le...,<h2>Current Events</h2><p>Brainiac 5 is presen...,{'api_detail_url': 'https://comicvine.gamespot...,1,1255,{'icon_url': 'https://comicvine.gamespot.com/a...,Brainiac 5,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Querl Dox,https://comicvine.gamespot.com/brainiac-5/4005...
3,Lyle Norg\nBrainiac 6,https://comicvine.gamespot.com/api/character/4...,None,427,2008-06-06 11:27:34,2020-02-12 18:19:27,"He is a member of the Legion of Super-Heroes, ...","<p data-right-indent=""0"" data-left-indent=""0"">...",{'api_detail_url': 'https://comicvine.gamespot...,1,1256,{'icon_url': 'https://comicvine.gamespot.com/a...,Invisible Kid,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Lyle Norg,https://comicvine.gamespot.com/invisible-kid/4...
4,Tinya Wazzo\nTinya Wazzo-Nah\nPhantom Woman\nP...,https://comicvine.gamespot.com/api/character/4...,None,761,2008-06-06 11:27:38,2020-10-09 07:42:05,Tinya Wazzo is a native of the extra-dimension...,"<h2>Origin</h2><figure data-align=""right"" data...",{'api_detail_url': 'https://comicvine.gamespot...,2,1257,{'icon_url': 'https://comicvine.gamespot.com/a...,Phantom Girl,{'api_detail_url': 'https://comicvine.gamespot...,{'api_detail_url': 'https://comicvine.gamespot...,Tinya Wazzo,https://comicvine.gamespot.com/phantom-girl/40...


In [291]:
spawn_response = requests.get(api_url + 'character/4005-3381?' + key_url + '&format=json', headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.0.1 Safari/605.1.15'})

In [292]:
character_response.status_code

200

In [293]:
spawn = json.loads(character_response.text)

In [301]:
spawn.keys()

dict_keys(['error', 'limit', 'offset', 'number_of_page_results', 'number_of_total_results', 'status_code', 'results', 'version'])

In [307]:
char_list = ['4005-24863', '4005-3381', '4005-40550', '4005-51840', '4005-42443', '4005-5210']
responses = []
for i in char_list:
    resp = requests.get(api_url+'character/'+str(i)+'?'+key_url + '&format=json', headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.0.1 Safari/605.1.15'} )
    chj = json.loads(resp.text)
    responses.append(chj['results'])

In [308]:
print(api_url + 'character/' + str(i) + '?' + key_url)

https://comicvine.gamespot.com/api/character/4005-5210?api_key=61e513aa5ef93ccd3df87a74b91f41a6fce71448


In [309]:
pd.DataFrame(responses)

,aliases,api_detail_url,birth,character_enemies,character_friends,count_of_issue_appearances,creators,date_added,date_last_updated,deck,...,origin,powers,publisher,real_name,site_detail_url,story_arc_credits,team_enemies,team_friends,teams,volume_credits
0,The Maxx\nBr'er Lapin,https://comicvine.gamespot.com/api/character/4...,None,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,126,[{'api_detail_url': 'https://comicvine.gamespo...,2008-06-06 11:27:35,2023-07-10 11:12:15,"A mysterious, foul smelling, cartoon loving, P...",...,{'api_detail_url': 'https://comicvine.gamespot...,[{'api_detail_url': 'https://comicvine.gamespo...,{'api_detail_url': 'https://comicvine.gamespot...,Dave,https://comicvine.gamespot.com/maxx/4005-24863/,[],[],[{'api_detail_url': 'https://comicvine.gamespo...,[],[{'api_detail_url': 'https://comicvine.gamespo...
1,Albert Francis Simmons\nLt. Colonel Al Simmons...,https://comicvine.gamespot.com/api/character/4...,None,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,1488,[{'api_detail_url': 'https://comicvine.gamespo...,2008-06-06 11:27:42,2024-08-17 07:14:45,"A highly trained assassin, murdered by his emp...",...,{'api_detail_url': 'https://comicvine.gamespot...,[{'api_detail_url': 'https://comicvine.gamespo...,{'api_detail_url': 'https://comicvine.gamespot...,Al Simmons,https://comicvine.gamespot.com/spawn-simmons/4...,[],[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...
2,Nolan Grayson,https://comicvine.gamespot.com/api/character/4...,None,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,156,[{'api_detail_url': 'https://comicvine.gamespo...,2008-06-06 11:27:34,2023-06-24 21:30:39,Father of the famous teen superhero Invincible...,...,{'api_detail_url': 'https://comicvine.gamespot...,[{'api_detail_url': 'https://comicvine.gamespo...,{'api_detail_url': 'https://comicvine.gamespot...,Nolan,https://comicvine.gamespot.com/omni-man/4005-4...,[],[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...
3,Dave Lizewski \r\nDave\r\nKA\r\nAssed Kicked\r...,https://comicvine.gamespot.com/api/character/4...,None,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,69,[{'api_detail_url': 'https://comicvine.gamespo...,2008-06-06 11:25:02,2019-05-27 14:01:21,Dave Lizewski was an average high school stude...,...,{'api_detail_url': 'https://comicvine.gamespot...,[{'api_detail_url': 'https://comicvine.gamespo...,{'api_detail_url': 'https://comicvine.gamespot...,David Lizewski,https://comicvine.gamespot.com/kick-ass/4005-5...,[],[],[],[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...
4,Officer Rick Grimes,https://comicvine.gamespot.com/api/character/4...,None,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,501,[{'api_detail_url': 'https://comicvine.gamespo...,2008-06-06 11:27:28,2023-06-23 07:17:18,Rick Grimes was formerly a police officer in t...,...,{'api_detail_url': 'https://comicvine.gamespot...,[{'api_detail_url': 'https://comicvine.gamespo...,{'api_detail_url': 'https://comicvine.gamespot...,Richard Grimes,https://comicvine.gamespot.com/rick-grimes/400...,[],[],[],[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...
5,"Mark Grayson,\nInvinciboy",https://comicvine.gamespot.com/api/character/4...,None,[{'api_detail_url': 'https://comicvine.gamespo...,[{'api_detail_url': 'https://comicvine.gamespo...,444,[{'api_detail_url': 'https://comicvine.gamespo...,2008-06-06 11:27:37,2024-03-02 06:00:2

I have begun to put the characters that I would like to focus on in their own dataframe so far. I did this by getting the ID's of each character manually and putting them in a list. The function that is 